In [ ]:
%pylab inline




# Example 2: Extracellular response of synaptic input
This is an example of **``LFPy``** running in an **``IPython notebook``**. To run through this example code and produce output, press **``<shift-Enter>``** in each code block below.

First step is to **import ``LFPy``** and other packages for analysis and plotting:

In [ ]:
import LFPy

Create some dictionarys with parameters for cell, synapse and extracellular electrode:

In [ ]:
cellParameters = {
    'morphology' : 'morphologies/L5_Mainen96_LFPy.hoc',
    'tstartms' : -50,
    'tstopms' : 100,
}

synapseParameters = {
    'syntype' : 'Exp2Syn',
    'e' : 0.,
    'tau1' : 0.5,
    'tau2' : 2.0,
    'weight' : 0.005,
    'record_current' : True,
}

z = mgrid[-400:1201:100]
electrodeParameters = {
    'x' : zeros(z.size),
    'y' : zeros(z.size),
    'z' : z,
    'sigma' : 0.3,
}

Then, create the **cell**, **synapse** and **electrode** objects using 
**LFPy.Cell**, **LFPy.Synapse**, **LFPy.RecExtElectrode**.

In [ ]:
cell = LFPy.Cell(**cellParameters)
cell.set_pos(xpos=-10, ypos=0, zpos=0)
cell.set_rotation(z=np.pi)

synapse = LFPy.Synapse(cell,
                       idx = cell.get_closest_idx(z=800),
                       **synapseParameters)
synapse.set_spike_times(array([10, 30, 50]))
                        
electrode = LFPy.RecExtElectrode(**electrodeParameters)

Run the simulation with **electrode** as input to **cell.simulate()**

In [ ]:
cell.simulate(electrode = electrode, rec_isyn=True)

Then plot the **somatic potential** and the **LFP**
from electrode object:

In [ ]:
figure(figsize=(12, 6))
subplot(133)
pcolormesh(cell.tvec, electrode.z, electrode.LFP,
           vmin=-abs(electrode.LFP).max(), vmax=abs(electrode.LFP).max(),
           cmap='spectral_r'), colorbar(), title('LFP (mV)')
subplot(232)
plot(cell.tvec, synapse.i), title('synaptic current (pA)')
subplot(235)
plot(cell.tvec, cell.somav), title('somatic voltage (mV)')
subplot(131)
for sec in LFPy.cell.neuron.h.allsec():
    idx = cell.get_idx(sec.name())
    plot(np.r_[cell.xstart[idx], cell.xend[idx][-1]],
            np.r_[cell.zstart[idx], cell.zend[idx][-1]],
            color='k')
plot([cell.synapses[0].x], [cell.synapses[0].z], \
     color=cell.synapses[0].color, marker=cell.synapses[0].marker, markersize=10)
plot(electrode.x, electrode.z, '.', marker='o', color='g')
axis([-500, 500, -400, 1200])

#savefig('LFPy-example-2.pdf', dpi=300)